In [7]:
%load_ext autoreload
%autoreload 2

In [5]:
from src import RGBNirBioDataModule, RGBNirBioModule
import pytorch_lightning as pl

dm = RGBNirBioDataModule(batch_size=700, num_workers=6, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
config = {
    "backbone": "resnet34",
    "pretrained": True,
    "optimizer": "Adam",
    "num_bio": 27,
    "bio_layers": [256, 512],
    "bio_dropout": 0.,
    "optimizer_params": {
        "lr": 1e-3
    },
    "early_stopping": False,
    "trainer": {
        "gpus": 1,
        "max_epochs": 7,
        "logger": None,
        "enable_checkpointing": True,
        "overfit_batches": 0,
        "deterministic": True,
        "precision": 16
    },
    "datamodule": {
        "batch_size": 700,
        "num_workers": 6,
        "pin_memory": True
    },
}

if __name__ == "__main__":
    model = RGBNirBioModule(config)
    #trainer = pl.Trainer(**config["trainer"], resume_from_checkpoint="checkpoints/r34rgbnir_da-val_loss=6.54345-epoch=3.ckpt")
    model = RGBNirBioModule.load_from_checkpoint("r34RGBNirBio_vl=6.35_epoch=7.ckpt")
    #trainer.fit(model, dm)
    model.hparams

In [5]:
from tqdm import tqdm

model.cuda()
dl = dm.test_dataloader()
labels, observations = [], []
for batch in tqdm(dl):
    preds = model.predict(batch)
    values, ixs = preds.topk(30)
    labels += [" ".join([str(i.item()) for i in ix]) for ix in ixs]
    observation_ids = batch["observation_id"]
    observations += observation_ids.numpy().tolist()
    

100%|██████████| 53/53 [01:09<00:00,  1.32s/it]


In [9]:
import pandas as pd
submission = pd.DataFrame({"Id": observations, "Predicted": labels})
#submission["Id"] = submission["Id"].astype(int)
submission.to_csv("submission.csv", index=False)
submission.sample(5)

NameError: name 'observations' is not defined

In [9]:
submission["Id"] = submission["Id"].astype(int)

In [10]:
submission.to_csv("submission.csv", index=False)
submission.sample(5)


,Id,Predicted
33609,21762543,3495 2902 5230 3132 5109 4949 5317 5068 5351 5...
6654,10742493,250 510 213 403 110 456 86 47 521 677 5 1090 4...
35155,21935529,3485 2902 6428 5317 5020 4870 4823 8084 4242 5...
1539,10066796,1041 472 336 751 263 600 9 692 660 211 748 380...
2149,10717747,373 316 399 334 85 333 235 250 319 140 754 592...


In [10]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

NameError: name 'submission' is not defined

In [11]:
len(sample_submission)

36421